# Treino XGBoost - Detecção de Fadiga

Treina modelo XGBoost para classificação binária: Alerta vs Sonolento

In [5]:
import numpy as np
import pandas as pd
import joblib
import json
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from scipy import stats
import xgboost as xgb

print("Bibliotecas carregadas")

Bibliotecas carregadas


In [6]:
# Carrega dados processados
dados_dir = Path("processed_data_uta_rldd_CORRECTED")
arquivo_X = dados_dir / "X_sequences.npy"
arquivo_y = dados_dir / "y_labels.npy"

if not arquivo_X.exists() or not arquivo_y.exists():
    print("Erro: Execute primeiro o notebook de preparação dos dados")
    print(f"Arquivos necessários:")
    print(f"  {arquivo_X}")
    print(f"  {arquivo_y}")
    raise FileNotFoundError("Arquivos de dados não encontrados")

# Carrega sequences e labels
X_sequences = np.load(arquivo_X)
y_labels = np.load(arquivo_y)

print(f"Dados carregados: {X_sequences.shape[0]} amostras")
print(f"Shape das sequences: {X_sequences.shape}")
print(f"Labels únicas: {np.unique(y_labels)}")

Dados carregados: 32111 amostras
Shape das sequences: (32111, 90, 4)
Labels únicas: [ 0  5 10]


In [7]:
# Converte para classificação binária
# 0,5 -> 0 (Alerta), 10 -> 1 (Sonolento)
y_binario = np.where(y_labels == 10, 1, 0)

print("Distribuição original:")
unique, counts = np.unique(y_labels, return_counts=True)
for label, count in zip(unique, counts):
    print(f"  {label}: {count} amostras")

print("\nDistribuição binária:")
unique, counts = np.unique(y_binario, return_counts=True)
labels_texto = ['Alerta', 'Sonolento']
for label, count, texto in zip(unique, counts, labels_texto):
    print(f"  {texto} ({label}): {count} amostras")

Distribuição original:
  0: 10826 amostras
  5: 10473 amostras
  10: 10812 amostras

Distribuição binária:
  Alerta (0): 21299 amostras
  Sonolento (1): 10812 amostras


In [8]:
# Extrai features estatísticas das sequences
class ExtratorFeatures:
    def __init__(self):
        nomes_sinais = ['PERCLOS', 'MAR', 'BLINK_RATE', 'HEAD_STABILITY']
        nomes_stats = ['mean', 'std', 'median', 'min', 'max', 'range', 'q25', 'q75', 'trend', 'zcr', 'autocorr']
        
        self.feature_names = []
        for sinal in nomes_sinais:
            for stat in nomes_stats:
                self.feature_names.append(f"{sinal}_{stat}")
    
    def trend_slope(self, sinal):
        if len(sinal) < 2:
            return 0.0
        x = np.arange(len(sinal))
        try:
            slope, _, _, _, _ = stats.linregress(x, sinal)
            return slope if not np.isnan(slope) else 0.0
        except:
            return 0.0
    
    def zero_crossing_rate(self, sinal):
        if len(sinal) < 2:
            return 0.0
        mean_centered = sinal - np.mean(sinal)
        crossings = np.sum(np.diff(np.sign(mean_centered)) != 0)
        return crossings / len(sinal)
    
    def autocorr_lag1(self, sinal):
        if len(sinal) < 3:
            return 0.0
        try:
            corr = np.corrcoef(sinal[:-1], sinal[1:])[0, 1]
            return corr if not np.isnan(corr) else 0.0
        except:
            return 0.0
    
    def extrair_features_sinal(self, sinal):
        features = [
            np.mean(sinal), np.std(sinal), np.median(sinal),
            np.min(sinal), np.max(sinal), np.ptp(sinal),
            np.percentile(sinal, 25), np.percentile(sinal, 75),
            self.trend_slope(sinal), self.zero_crossing_rate(sinal), self.autocorr_lag1(sinal)
        ]
        return features
    
    def transform(self, X_sequences):
        n_samples = X_sequences.shape[0]
        n_features = len(self.feature_names)
        X_features = np.zeros((n_samples, n_features))
        
        for i in range(n_samples):
            sequence = X_sequences[i]
            sample_features = []
            
            for signal_idx in range(4):
                sinal = sequence[:, signal_idx]
                features_sinal = self.extrair_features_sinal(sinal)
                sample_features.extend(features_sinal)
            
            X_features[i] = sample_features
        
        return X_features

extrator = ExtratorFeatures()
print(f"Extrator criado - {len(extrator.feature_names)} features por amostra")

Extrator criado - 44 features por amostra


In [9]:
# Extrai features e divide dados
print("Extraindo features...")
X_features = extrator.transform(X_sequences)
print(f"Features extraídas: {X_features.shape}")

# Divide treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y_binario, test_size=0.2, random_state=42, stratify=y_binario
)

print(f"Treino: {X_train.shape[0]} amostras")
print(f"Teste: {X_test.shape[0]} amostras")

Extraindo features...


/home/zhizhu/tcc/ia-fadiga-main2 (copy)/fatigue_env/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/zhizhu/tcc/ia-fadiga-main2 (copy)/fatigue_env/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Features extraídas: (32111, 44)
Treino: 25688 amostras
Teste: 6423 amostras


In [10]:
# Normaliza features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features normalizadas")

Features normalizadas


In [11]:
# Treina XGBoost
print("Treinando XGBoost...")

# Calcula pesos das classes
unique, counts = np.unique(y_train, return_counts=True)
total = len(y_train)
peso_alerta = total / (2 * counts[0]) if 0 in unique else 1.0
peso_sonolento = total / (2 * counts[1]) if 1 in unique else 1.0

print(f"Peso Alerta: {peso_alerta:.2f}")
print(f"Peso Sonolento: {peso_sonolento:.2f}")

modelo_xgb = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    scale_pos_weight=peso_sonolento/peso_alerta,
    random_state=42,
    eval_metric='logloss'
)

modelo_xgb.fit(X_train_scaled, y_train)
print("Modelo treinado!")

Treinando XGBoost...
Peso Alerta: 0.75
Peso Sonolento: 1.49
Modelo treinado!


In [12]:
# Avalia modelo
y_pred = modelo_xgb.predict(X_test_scaled)
y_proba = modelo_xgb.predict_proba(X_test_scaled)

print("\nResultados:")
print(classification_report(y_test, y_pred, target_names=['Alerta', 'Sonolento']))

print("\nMatriz de Confusão:")
cm = confusion_matrix(y_test, y_pred)
print(f"         Alerta  Sonolento")
print(f"Alerta     {cm[0,0]:4d}      {cm[0,1]:4d}")
print(f"Sonolento  {cm[1,0]:4d}      {cm[1,1]:4d}")

accuracy = (cm[0,0] + cm[1,1]) / cm.sum()
print(f"\nAcurácia: {accuracy:.3f}")


Resultados:
              precision    recall  f1-score   support

      Alerta       0.92      0.88      0.90      4260
   Sonolento       0.79      0.85      0.82      2163

    accuracy                           0.87      6423
   macro avg       0.85      0.87      0.86      6423
weighted avg       0.87      0.87      0.87      6423


Matriz de Confusão:
         Alerta  Sonolento
Alerta     3758       502
Sonolento   327      1836

Acurácia: 0.871


In [ ]:
# Salva modelo e componentes
modelo_dir = Path("modelos_xgb")
modelo_dir.mkdir(exist_ok=True)

# Salva modelo
joblib.dump(modelo_xgb, modelo_dir / "modelo_xgb.joblib")

# Salva scaler
joblib.dump(scaler, modelo_dir / "scaler.joblib")

# Não salva extrator - será criado no pipeline

# Info das classes
class_info = {
    "classes": [0, 1],
    "class_names": ["Alerta", "Sonolento"],
    "accuracy": float(accuracy)
}

with open(modelo_dir / "info_classes.json", 'w') as f:
    json.dump(class_info, f, indent=2)

print(f"Modelo salvo em: {modelo_dir}")
print("Arquivos criados:")
print("  - modelo_xgb.joblib")
print("  - scaler.joblib")
print("  - info_classes.json")

In [14]:
# Cria pipeline de predição
pipeline_code = '''import joblib
import numpy as np
import json
from pathlib import Path

class PipelineFadiga:
    def __init__(self, modelo_dir):
        modelo_dir = Path(modelo_dir)
        
        self.extrator = joblib.load(modelo_dir / "extrator.joblib")
        self.scaler = joblib.load(modelo_dir / "scaler.joblib")
        self.modelo = joblib.load(modelo_dir / "modelo_xgb.joblib")
        
        with open(modelo_dir / "info_classes.json", "r") as f:
            self.class_info = json.load(f)
    
    def predict_sequence(self, sequence):
        """Prediz fadiga de uma sequence (90, 4)"""
        sequences = np.expand_dims(sequence, axis=0)
        features = self.extrator.transform(sequences)
        features_scaled = self.scaler.transform(features)
        
        prediction = self.modelo.predict(features_scaled)[0]
        probabilities = self.modelo.predict_proba(features_scaled)[0]
        
        prob_dict = {
            "Alerta": float(probabilities[0]),
            "Sonolento": float(probabilities[1])
        }
        
        class_name = "Alerta" if prediction == 0 else "Sonolento"
        
        return prediction, prob_dict, class_name
'''

with open(modelo_dir / "pipeline.py", 'w') as f:
    f.write(pipeline_code)

print("Pipeline de predição criado: pipeline.py")
print("\nTreino concluído! Execute o notebook de teste em tempo real.")

Pipeline de predição criado: pipeline.py

Treino concluído! Execute o notebook de teste em tempo real.
